# Lecture Objectives
brought to you by Kayla Thomas, Elliot Cohen, Adam Richards, Frank Burkholder, and Erich Wellinger.

- Connect to a database from within a python program and run queries
- Understand psycopg2's cursors and commits
- Generate dynamic queries

### Combining SQL and Python

You'll find yourself working with data that are only accessable through SQL.  However, your machine-learning capabilities are built in Python.  To resolve this, set up a connection from Python to the SQL database to bring in the data.

### Why do we care?

- SQL-based databases are common in industry environments
- Can leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
- Useful for data pipelines, pre-cleaning, data exploration
- Allows for dynamic query generation and hence automation

### psycopg2

- A Python library that connects and interacts with PostgresSQL databases.
- psycopg2 is a wrapper around libpq (C application programmer's interface) that exposes a Python DB-API to Python programs.
- [Documentation--Includes Installation Instructions](http://initd.org/psycopg/docs/install.html)
- In addition to what's listed in the documentation, if you have the anaconda distribution of Python 
```bash 
$ conda install psycopg2 
```
should work.
- There are similar packages for other flavors of SQL that work much the same way:
  
    [msql.connector](https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html)  
    [pyodbc](https://github.com/mkleehammer/pyodbc)

### General Workflow

1. Establish a connection to Postgres database using psycopg2
2. Create a [cursor](http://initd.org/psycopg/docs/cursor.html#cursor)
3. Use the cursor to execute SQL queries and retrieve data
4. Commit SQL actions
4. Close the cursor and connection

## First create a database from an admin account

Database creation should be reserved for only administrators.  Each database should have a list of non-admin users that are specific to that database.  Keeping this separation of roles is a setup that helps with security. 

### Database setup

Type the commands below in bash (indicated by being prefaced with $). 

However, on **MacOS use only the second command**.

```bash
$ docker exec -it pgserv bash
# psql -U postgres
=# CREATE USER ender WITH ENCRYPTED PASSWORD 'bugger';
=# CREATE DATABASE golf WITH OWNER ender;
=# \q
```
Now type the following commands in postgres (indicated by being prefaced with #).

```sql
# CREATE USER ender WITH ENCRYPTED PASSWORD 'bugger';
# CREATE DATABASE golf WITH OWNER ender;
```

Check to see if you see the new database and other useful commands

```sql
# \list    # lists all the databases in Postgres
# \connect # connect to a specific database
# \dt      # list tables in the currently connected database
# \q       # quit 
```

### Connect to the database
- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you need to create a database, you can first connect to Postgres using the dbname 'postgres' to initialize

In [30]:
import psycopg2
import getpass

upass = getpass.getpass()
conn = psycopg2.connect(database="golf", user="ender", password=upass, host="localhost", port="5435") #port="5432"
print("connected")

 ······


connected


In [8]:
conn

<connection object at 0x7f0e39d06c20; dsn: 'user=ender password=xxx dbname=golf host=localhost port=5435', closed: 0>

### Instantiate the Cursor

- A cursor is a control structure that enables traversal over the records in a database
- Executes and fetches data
- When the cursor points at the resulting output of a query, it can only read each observation once.  If you choose to see a previously read observation, you must rerun the query. 
- Can be closed without closing the connection

In [9]:
cur = conn.cursor()

### Commits

- Data changes are not actually stored until you choose to commit
- You can choose to have automatic commit by using ` autocommit = True`
- When connecting directly to the Postgres Server to initiate server level commands such as creating a database, you must use the `autocommit = True` option since Postgres does not have "temporary" transactions at the database level

### Disconnect from the cursor and database
- Cursors and Connections must be closed using .close() or else Postgres will lock certain operations on the database/tables. 

# Populate the database

You'll need to enter your password again when you run the script.

In [10]:
run populate.py

 ······


connected
(datetime.date(2014, 7, 1), 'sunny', 85, 85, False, "Don't Play")
(datetime.date(2014, 7, 2), 'sunny', 80, 90, True, "Don't Play")
(datetime.date(2014, 7, 3), 'overcast', 83, 78, False, 'Play')
(datetime.date(2014, 7, 4), 'rain', 70, 96, False, 'Play')
(datetime.date(2014, 7, 5), 'rain', 68, 80, False, 'Play')
(datetime.date(2014, 7, 6), 'rain', 65, 70, True, "Don't Play")
(datetime.date(2014, 7, 7), 'overcast', 64, 65, True, 'Play')
(datetime.date(2014, 7, 8), 'sunny', 72, 95, False, "Don't Play")
(datetime.date(2014, 7, 9), 'sunny', 69, 70, False, 'Play')
(datetime.date(2014, 7, 10), 'rain', 75, 80, False, 'Play')
(datetime.date(2014, 7, 11), 'sunny', 75, 70, True, 'Play')
(datetime.date(2014, 7, 12), 'overcast', 72, 90, True, 'Play')
(datetime.date(2014, 7, 13), 'overcast', 81, 75, False, 'Play')
(datetime.date(2014, 7, 14), 'rain', 71, 80, True, "Don't Play")


You should be seeing 14 lines of data above.

### Create a new table

**you may need to edit the populate.py script with your username and password**

The populate.py script does the following:

1. Connects to the database
2. Checks to see if a 'golf' table exists and if so drops it
3. Creates a 'golf' table
4. Populates the golf table with 'playgolf.csv'
5. Queries the table for the first 30 items


### Run a query to get 30 records from our data

In [11]:
cur = conn.cursor()
query = '''
        SELECT *
        FROM golf
        LIMIT 30;
        '''

cur.execute(query)

### Lets look at our data one line at a time

In [12]:
cur.fetchone()

(datetime.date(2014, 7, 1), 'sunny', 85, 85, False, "Don't Play")

### Many lines at a time

In [13]:
#fetchmany(n) to get n rows
cur.fetchmany(5)

[(datetime.date(2014, 7, 2), 'sunny', 80, 90, True, "Don't Play"),
 (datetime.date(2014, 7, 3), 'overcast', 83, 78, False, 'Play'),
 (datetime.date(2014, 7, 4), 'rain', 70, 96, False, 'Play'),
 (datetime.date(2014, 7, 5), 'rain', 68, 80, False, 'Play'),
 (datetime.date(2014, 7, 6), 'rain', 65, 70, True, "Don't Play")]

### Or everything at once

But are there any results left?  With the cursor we only see them once and then the cursor moves on...

In [14]:
results = cur.fetchall() # fetchall() grabs all remaining rows

In [15]:
type(results)

list

In [16]:
type(results[0])

tuple

In [17]:
results

[(datetime.date(2014, 7, 7), 'overcast', 64, 65, True, 'Play'),
 (datetime.date(2014, 7, 8), 'sunny', 72, 95, False, "Don't Play"),
 (datetime.date(2014, 7, 9), 'sunny', 69, 70, False, 'Play'),
 (datetime.date(2014, 7, 10), 'rain', 75, 80, False, 'Play'),
 (datetime.date(2014, 7, 11), 'sunny', 75, 70, True, 'Play'),
 (datetime.date(2014, 7, 12), 'overcast', 72, 90, True, 'Play'),
 (datetime.date(2014, 7, 13), 'overcast', 81, 75, False, 'Play'),
 (datetime.date(2014, 7, 14), 'rain', 71, 80, True, "Don't Play")]

### You can even iterate over the cursor

In [18]:
cur.execute(query)
for record in cur:
    print ("date:{}, outlook:{}, temperature:{}".format(record[0], record[1], record[2]))

date:2014-07-01, outlook:sunny, temperature:85
date:2014-07-02, outlook:sunny, temperature:80
date:2014-07-03, outlook:overcast, temperature:83
date:2014-07-04, outlook:rain, temperature:70
date:2014-07-05, outlook:rain, temperature:68
date:2014-07-06, outlook:rain, temperature:65
date:2014-07-07, outlook:overcast, temperature:64
date:2014-07-08, outlook:sunny, temperature:72
date:2014-07-09, outlook:sunny, temperature:69
date:2014-07-10, outlook:rain, temperature:75
date:2014-07-11, outlook:sunny, temperature:75
date:2014-07-12, outlook:overcast, temperature:72
date:2014-07-13, outlook:overcast, temperature:81
date:2014-07-14, outlook:rain, temperature:71


# Dynamic Queries

- A Dynamic Query is a query that facilitates automated generation of program statements.  It's flexible based on context.

### First lets get an idea of how many records we start with

In [19]:
cur.execute('SELECT count(*) FROM golf;')
record_count = cur.fetchone()

In [20]:
record_count

(14,)

In [21]:
record_count = record_count[0]
record_count

14

### Create a query template and determine file path for imports

Use string formatting to generate a query for each approved file.

**[WARNING: BEWARE OF SQL INJECTION](http://initd.org/psycopg/docs/usage.html)**

NEVER use + or % or .format to reformat strings to be used with .execute

In [22]:
num = 579
terribly_unsafe = "SELECT * FROM golf WHERE userid = {num}; ".format(num=num)
print (terribly_unsafe)


date_cut = "2014-08-01"
horribly_risky = "SELECT * FROM golf WHERE tmstmp > %s;" % date_cut
print (horribly_risky)

SELECT * FROM golf WHERE userid = 579; 
SELECT * FROM golf WHERE tmstmp > 2014-08-01;


**Q** Why is this unsafe?  
**A** Python is happy, but if `num` or `date_cut` included something malicious your data could be at risk

### What is a SQL Injection Attack?

In [23]:
date_cut = "2014-08-01; DROP TABLE logins" # The user enters a date in a field on a web form
horribly_risky = "SELECT * FROM logins WHERE tmstmp > {};".format(date_cut)
print (horribly_risky)

SELECT * FROM logins WHERE tmstmp > 2014-08-01; DROP TABLE logins;


### Practice safe SQL with Psycopg2
```
>>> sql_query = "INSERT INTO authors (name) VALUES (%s);" 
>>> data = ("O'Reilly", )
>>> cur.execute(sql_query, data) # Note: no % operator
```

<img src='images/Injection.PNG' width='600'></img>

### Visit [bobby-tables.com](http://www.bobby-tables.com/) to learn more about SQL safety.



### Psygopg2 can leave databases "idle in transaction"  
To prevent this **commit** or **rollback** the transaction then **close** it.

In [31]:
conn.commit() # commits whatever changed you made to the database

In [33]:
conn.rollback() # rollsback (undo) present pending transaction

In [35]:
conn.close() # closes the connection

## `with` statements

In [27]:
query = "SELECT count(*) FROM golf;"
upass = getpass.getpass()

with psycopg2.connect(database="golf", user="ender", password=upass, host="localhost", port="5435") as conn: #port="5432"
    with conn.cursor() as curs:
        print("Cursor inside with block: {}".format(curs))
        curs.execute(query)
        print("Cursor closed? {}".format(curs.closed))
    print("Cursor outside with block: {}".format(curs))
    print("Cursor closed? {}".format(curs.closed))
    print("Connection closed? {}".format(conn.closed))

 ······


Cursor inside with block: <cursor object at 0x7f0e48482b80; closed: 0>
Cursor closed? False
Cursor outside with block: <cursor object at 0x7f0e48482b80; closed: -1>
Cursor closed? True
Connection closed? 0


### The connection is *not* closed automatically:

In [36]:
conn

<connection object at 0x7f0e39d0ec20; dsn: 'user=ender password=xxx dbname=golf host=localhost port=5435', closed: 1>

In [29]:
conn.close()
print("Connection closed? {}".format(conn.closed))
conn

Connection closed? 1


<connection object at 0x7f0e39d0e040; dsn: 'user=ender password=xxx dbname=golf host=localhost port=5435', closed: 1>

# Review

* Connections must be established using an existing database, username, database IP/URL, and maybe passwords
* If you have no created databases, you can connect to Postgres using the dbname 'postgres' to initialize db commands
* Data changes are not actually stored until you choose to commit. This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions is only temporary stored.
* Autocommit = True is necessary to do database commands like CREATE DATABASE.  This is because Postgres does not have temporary transactions at the database level.
* If you ever need to build similar pipelines for other forms of database, there are libraries such PyODBC which operate very similarly.
* SQL connection databases utilizes cursors for data traversal and retrieval.  This is kind of like an iterator in Python.
* Cursor operations typically goes like the following:
    - execute a query
    - fetch rows from query result if it is a SELECT query
    - because it is iterative, previously fetched rows can only be fetched again by rerunning the query
    - close cursor through .close()
* Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables until the connection is severed. 

## Additional Resources

<table border="1" class="data-types docutils">
<caption>Default mapping between Python and PostgreSQL types</caption>
<colgroup>
<col width="28%">
<col width="35%">
<col width="37%">
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Python</th>
<th class="head">PostgreSQL</th>
<th class="head">See also</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><code class="xref py py-obj docutils literal"><span class="pre">None</span></code></td>
<td><code class="sql docutils literal"><span class="pre">NULL</span></code></td>
<td rowspan="2"><a class="reference internal" href="#adapt-consts"><span class="std std-ref">Constants adaptation</span></a></td>
</tr>
<tr class="row-odd"><td><code class="xref py py-obj docutils literal"><span class="pre">bool</span></code></td>
<td><code class="sql docutils literal"><span class="pre">bool</span></code></td>
</tr>
<tr class="row-even"><td><code class="xref py py-obj docutils literal"><span class="pre">float</span></code></td>
<td><div class="first last line-block">
<div class="line"><code class="sql docutils literal"><span class="pre">real</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">double</span></code></div>
</div>
</td>
<td rowspan="3"><a class="reference internal" href="#adapt-numbers"><span class="std std-ref">Numbers adaptation</span></a></td>
</tr>
<tr class="row-odd"><td><div class="first last line-block">
<div class="line"><code class="xref py py-obj docutils literal"><span class="pre">int</span></code></div>
<div class="line"><code class="xref py py-obj docutils literal"><span class="pre">long</span></code></div>
</div>
</td>
<td><div class="first last line-block">
<div class="line"><code class="sql docutils literal"><span class="pre">smallint</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">integer</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">bigint</span></code></div>
</div>
</td>
</tr>
<tr class="row-even"><td><a class="reference external" href="https://docs.python.org/3/library/decimal.html#decimal.Decimal" title="(in Python v3.6)"><code class="xref py py-obj docutils literal"><span class="pre">Decimal</span></code></a></td>
<td><code class="sql docutils literal"><span class="pre">numeric</span></code></td>
</tr>
<tr class="row-odd"><td><div class="first last line-block">
<div class="line"><code class="xref py py-obj docutils literal"><span class="pre">str</span></code></div>
<div class="line"><code class="xref py py-obj docutils literal"><span class="pre">unicode</span></code></div>
</div>
</td>
<td><div class="first last line-block">
<div class="line"><code class="sql docutils literal"><span class="pre">varchar</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">text</span></code></div>
</div>
</td>
<td><a class="reference internal" href="#adapt-string"><span class="std std-ref">Strings adaptation</span></a></td>
</tr>
<tr class="row-even"><td><div class="first last line-block">
<div class="line"><a class="reference external" href="https://docs.python.org/2/library/functions.html#buffer" title="(in Python v2.7)"><code class="xref py py-obj docutils literal"><span class="pre">buffer</span></code></a></div>
<div class="line"><a class="reference external" href="https://docs.python.org/3/library/stdtypes.html#memoryview" title="(in Python v3.6)"><code class="xref py py-obj docutils literal"><span class="pre">memoryview</span></code></a></div>
<div class="line"><a class="reference external" href="https://docs.python.org/3/library/stdtypes.html#bytearray" title="(in Python v3.6)"><code class="xref py py-obj docutils literal"><span class="pre">bytearray</span></code></a></div>
<div class="line"><a class="reference external" href="https://docs.python.org/3/library/stdtypes.html#bytes" title="(in Python v3.6)"><code class="xref py py-obj docutils literal"><span class="pre">bytes</span></code></a></div>
<div class="line">Buffer protocol</div>
</div>
</td>
<td><code class="sql docutils literal"><span class="pre">bytea</span></code></td>
<td><a class="reference internal" href="#adapt-binary"><span class="std std-ref">Binary adaptation</span></a></td>
</tr>
<tr class="row-odd"><td><code class="xref py py-obj docutils literal"><span class="pre">date</span></code></td>
<td><code class="sql docutils literal"><span class="pre">date</span></code></td>
<td rowspan="4"><a class="reference internal" href="#adapt-date"><span class="std std-ref">Date/Time objects adaptation</span></a></td>
</tr>
<tr class="row-even"><td><code class="xref py py-obj docutils literal"><span class="pre">time</span></code></td>
<td><div class="first last line-block">
<div class="line"><code class="sql docutils literal"><span class="pre">time</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">timetz</span></code></div>
</div>
</td>
</tr>
<tr class="row-odd"><td><code class="xref py py-obj docutils literal"><span class="pre">datetime</span></code></td>
<td><div class="first last line-block">
<div class="line"><code class="sql docutils literal"><span class="pre">timestamp</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">timestamptz</span></code></div>
</div>
</td>
</tr>
<tr class="row-even"><td><code class="xref py py-obj docutils literal"><span class="pre">timedelta</span></code></td>
<td><code class="sql docutils literal"><span class="pre">interval</span></code></td>
</tr>
<tr class="row-odd"><td><code class="xref py py-obj docutils literal"><span class="pre">list</span></code></td>
<td><code class="sql docutils literal"><span class="pre">ARRAY</span></code></td>
<td><a class="reference internal" href="#adapt-list"><span class="std std-ref">Lists adaptation</span></a></td>
</tr>
<tr class="row-even"><td><div class="first last line-block">
<div class="line"><code class="xref py py-obj docutils literal"><span class="pre">tuple</span></code></div>
<div class="line"><code class="xref py py-obj docutils literal"><span class="pre">namedtuple</span></code></div>
</div>
</td>
<td><div class="first last line-block">
<div class="line">Composite types</div>
<div class="line"><code class="sql docutils literal"><span class="pre">IN</span></code> syntax</div>
</div>
</td>
<td><div class="first last line-block">
<div class="line"><a class="reference internal" href="#adapt-tuple"><span class="std std-ref">Tuples adaptation</span></a></div>
<div class="line"><a class="reference internal" href="extras.html#adapt-composite"><span class="std std-ref">Composite types casting</span></a></div>
</div>
</td>
</tr>
<tr class="row-odd"><td><code class="xref py py-obj docutils literal"><span class="pre">dict</span></code></td>
<td><code class="sql docutils literal"><span class="pre">hstore</span></code></td>
<td><a class="reference internal" href="extras.html#adapt-hstore"><span class="std std-ref">Hstore data type</span></a></td>
</tr>
<tr class="row-even"><td>Psycopg’s <code class="xref py py-obj docutils literal"><span class="pre">Range</span></code></td>
<td><code class="sql docutils literal"><span class="pre">range</span></code></td>
<td><a class="reference internal" href="extras.html#adapt-range"><span class="std std-ref">Range data types</span></a></td>
</tr>
<tr class="row-odd"><td>Anything™</td>
<td><code class="sql docutils literal"><span class="pre">json</span></code></td>
<td><a class="reference internal" href="extras.html#adapt-json"><span class="std std-ref">JSON adaptation</span></a></td>
</tr>
<tr class="row-even"><td><a class="reference external" href="https://docs.python.org/3/library/uuid.html#uuid.UUID" title="(in Python v3.6)"><code class="xref py py-obj docutils literal"><span class="pre">UUID</span></code></a></td>
<td><code class="sql docutils literal"><span class="pre">uuid</span></code></td>
<td><a class="reference internal" href="extras.html#adapt-uuid"><span class="std std-ref">UUID data type</span></a></td>
</tr>
<tr class="row-odd"><td><a class="reference external" href="https://docs.python.org/3/library/ipaddress.html#module-ipaddress" title="(in Python v3.6)"><code class="xref py py-obj docutils literal"><span class="pre">ipaddress</span></code></a>
objects</td>
<td><div class="first last line-block">
<div class="line"><code class="sql docutils literal"><span class="pre">inet</span></code></div>
<div class="line"><code class="sql docutils literal"><span class="pre">cidr</span></code></div>
</div>
</td>
<td><a class="reference internal" href="extras.html#adapt-network"><span class="std std-ref">Networking data types</span></a></td>
</tr>
</tbody>
</table>